<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Example-Together" data-toc-modified-id="Example-Together-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example Together</a></span><ul class="toc-item"><li><span><a href="#Question" data-toc-modified-id="Question-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Question</a></span></li><li><span><a href="#Considerations" data-toc-modified-id="Considerations-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Considerations</a></span></li><li><span><a href="#Loading-the-Data" data-toc-modified-id="Loading-the-Data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Loading the Data</a></span></li><li><span><a href="#Some-Exploration-to-Better-Understand-our-Data" data-toc-modified-id="Some-Exploration-to-Better-Understand-our-Data-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Some Exploration to Better Understand our Data</a></span></li><li><span><a href="#Experimental-Setup" data-toc-modified-id="Experimental-Setup-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Experimental Setup</a></span><ul class="toc-item"><li><span><a href="#What-Test-Would-Make-Sense?" data-toc-modified-id="What-Test-Would-Make-Sense?-2.5.1"><span class="toc-item-num">2.5.1&nbsp;&nbsp;</span>What Test Would Make Sense?</a></span></li><li><span><a href="#The-Hypotheses" data-toc-modified-id="The-Hypotheses-2.5.2"><span class="toc-item-num">2.5.2&nbsp;&nbsp;</span>The Hypotheses</a></span></li><li><span><a href="#Setting-a-Threshold" data-toc-modified-id="Setting-a-Threshold-2.5.3"><span class="toc-item-num">2.5.3&nbsp;&nbsp;</span>Setting a Threshold</a></span></li></ul></li><li><span><a href="#$\chi^2$-Test" data-toc-modified-id="$\chi^2$-Test-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>$\chi^2$ Test</a></span><ul class="toc-item"><li><span><a href="#Setup-the-Data" data-toc-modified-id="Setup-the-Data-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span>Setup the Data</a></span></li><li><span><a href="#Calculation" data-toc-modified-id="Calculation-2.6.2"><span class="toc-item-num">2.6.2&nbsp;&nbsp;</span>Calculation</a></span></li></ul></li><li><span><a href="#Interpretation" data-toc-modified-id="Interpretation-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Interpretation</a></span></li></ul></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exercise</a></span></li></ul></div>

In [2]:
import numpy as np
import pandas as pd

from scipy import stats
import seaborn as sns

# Objectives

- Conduct an A/B test in Python
- Interpret the results of the A/B tests for a stakeholder

# Example Together

## Question

We have data about whether customers completed sales transactions, segregated by the type of ad banners to which the customers were exposed.

The question we want to answer is whether there was any difference in sales "conversions" between desktop customers who saw the sneakers banner and desktop customers who saw the accessories banner in the month of May 2019.

## Considerations

What would we need to consider when designing our experiment?

Might include:

- Who is it that we're including in our test?
- How big of an effect would make it "worth" us seeing?
    - This can affect sample size
    - This can give context of a statistically significant result
- Other biases or "gotchas"

## Loading the Data

First let's download the data from [kaggle](https://www.kaggle.com/podsyp/how-to-do-product-analytics) via the release page of this repo: https://github.com/flatiron-school/ds-ab_testing/releases 

The code below will load it into our DataFrame:

In [3]:
# This will download the data from online so it can take
# some time (but relatively small download)

df = pd.read_csv('https://github.com/flatiron-school/ds-ab_testing/\
releases/download/v1.2/products_small.csv')

> Let's take a look while we're at it

In [ ]:
# Check it out


In [ ]:
# All the deets


## Some Exploration to Better Understand our Data

Lets's look at the different banner types:

In [4]:
# banner types
df['product'].value_counts()

clothes             210996
company             203020
sneakers            201298
sports_nutrition    193200
accessories         191486
Name: product, dtype: int64

In [5]:
df.site_version.value_counts()

mobile     718521
desktop    281479
Name: site_version, dtype: int64

In [6]:
# products and targets
df.groupby('product')['target'].value_counts(normalize=True)

product           target
accessories       0         0.972014
                  1         0.027986
clothes           0         0.937079
                  1         0.062921
company           0         1.000000
sneakers          0         0.960640
                  1         0.039360
sports_nutrition  0         0.985316
                  1         0.014684
Name: target, dtype: float64

Let's look at the range of time-stamps on these data:

In [7]:
# Date range
df['time'].min()

'2019-01-01 00:00:25'

In [8]:
df['time'].max()

'2019-05-31 23:59:21'

Let's check the counts of the different site_version values:

In [ ]:
# Site types


In [12]:
# action title
df['title'].value_counts()

banner_show     872275
banner_click     98330
order            29395
Name: title, dtype: int64

In [10]:
# double check
df['target'].value_counts()

0    970605
1     29395
Name: target, dtype: int64

In [13]:
len(df['title'].loc[df['title'] == 'order'])

29395

In [14]:
df.groupby('title').count()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,target
title,,,,,,,,
banner_click,98330,98330,98330,98330,98330,98330,98330,98330
banner_show,872275,872275,872275,872275,872275,872275,872275,872275
order,29395,29395,29395,29395,29395,29395,29395,29395


In [17]:
#This one here
order_filter = df['title'] == 'order'

target_filter = df['target'] == 1

df.loc[order_filter & target_filter].shape

(29395, 9)

## Experimental Setup

We need to filter by site_version, time, and product:

In [19]:
desk_filter = df['site_version'] == 'desktop'
time_filter = df['time'] > '2019-05-01'
product_filter = df['product'].isin(['sneakers', 'accessories'])
#


In [20]:
# sniff test!
df2 = df.loc[desk_filter & time_filter & product_filter]

In [25]:
df2['product'].value_counts()

sneakers       13269
accessories    12703
Name: product, dtype: int64

### What Test Would Make Sense?

Since we're comparing the frequency of conversions of customers who saw the "sneakers" banner against those who saw the "accessories" banner, we can use a $\chi^2$ test.

Note there are other hypothesis tests we can use but this should be fine since it should fit our criteria.

### The Hypotheses

$H_0$: frequency of conversions are the same

$H_1$: not null

### Setting a Threshold

We'll set a false-positive rate of $\alpha = 0.05$.

## $\chi^2$ Test

### Setup the Data

We need our contingency table: the numbers of people who did or did not submit orders, both for the accessories banner and the sneakers banner. 

In [26]:
# Create the two product types
df_A = df2.loc[df2['product'] == 'accessories']
df_B = df2.loc[df2['product'] == 'sneakers']

In [28]:
# Get number of orders
accessories_orders = sum(df_A['target'])
sneakers_orders = sum(df_B['target'])

In [29]:
accessories_orders, sneakers_orders

(496, 799)

To get the numbers of people who didn't submit orders, we get the total number of people who were shown banners and then subtract the numbers of people who did make orders.

In [38]:
# get conversion rates
accessories_total = len(df_A)
sneakers_total = len(df_B)

accessories_no_order = accessories_total - accessories_orders
sneakers_no_order = sneakers_total - sneakers_orders

In [39]:
accessories_no_order, sneakers_no_order

(12207, 12470)

In [41]:
# Create table
contingency_table = np.array([
    [accessories_orders, accessories_no_order],
    [sneakers_orders, sneakers_no_order]
])

In [42]:
contingency_table

array([[  496, 12207],
       [  799, 12470]])

### Calculation


In [43]:
# Chi^2!

stats.chi2_contingency(contingency_table)

(60.94640624666488,
 5.86498985882812e-15,
 1,
 array([[  633.38922686, 12069.61077314],
        [  661.61077314, 12607.38922686]]))

This extremely low $p$-value suggests that these two groups are genuinely performing differently. In particular, the desktop customers who saw the sneakers banner in May 2019 bought at a higher rate than the desktop customers who saw the accessories banner in May 2019.

## Interpretation

In [44]:
contingency_table

array([[  496, 12207],
       [  799, 12470]])

In [45]:
# Find the difference in conversion rate
accessory_CR, sneaker_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)

In [46]:
print(f'Conversion Rate for accessory banner:\n\t{100*accessory_CR:.3f}%')
print(f'Conversion Rate for sneaker banner:\n\t{100*sneaker_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(sneaker_CR-accessory_CR):.3f}%')

Conversion Rate for accessory banner:
	3.905%
Conversion Rate for sneaker banner:
	6.022%

Absolute difference of CR: 2.117%



So we can say:
- There was a statistically significant difference at the $\alpha$-level (confidence level)
- The difference was about $2.1\%$ in favor of the sneaker banner!

# Exercise

> The company is impressed with what you found and is now wondering if there is a difference in their other banner ads!

With your group, look at the same month (May 2019) but compare different platforms ('mobile' vs 'desktop') and or different banner types ('accessories', 'sneakers', 'clothes', 'sports_nutrition'). Just don't repeat the same test we did above 😉

Make sure you record what considerations you have for the experiment, what hypothesis test you performed ($H_0$ and $H_1$ too), and your overall conclusion/interpretation for the _business stakeholders_. Is there a follow-up you'd suggest? 

In [47]:
platform_filter = df['site_version'].isin(['desktop', 'mobile'])
time_filter = df['time'] > '2019-05-01'
product_filter = df['product'].isin(['sneakers', 'accessories', 'clothes', 'sports_nutrition'])

In [48]:
df3 = df.loc[platform_filter & time_filter & product_filter]

In [50]:
df3['product'].value_counts()

clothes             53451
sneakers            51194
sports_nutrition    50508
accessories         48165
Name: product, dtype: int64

In [51]:
df_desktop = df3.loc[df3['site_version'] == 'desktop']
df_mobile =  df3.loc[df3['site_version'] == 'mobile']

In [53]:
df_mobile['product'].value_counts()

clothes             38373
sneakers            37925
sports_nutrition    37338
accessories         35462
Name: product, dtype: int64

In [54]:
df_desktop['product'].value_counts()

clothes             15078
sneakers            13269
sports_nutrition    13170
accessories         12703
Name: product, dtype: int64

In [60]:
mobile_accessories_total = len(df_mobile.loc[df_mobile['product'] == 'accessories'])
mobile_sneakers_total = len(df_mobile.loc[df_mobile['product'] == 'sneakers'])
mobile_clothes_total = len(df_mobile.loc[df_mobile['product'] == 'clothes'])
mobile_sn_total = len(df_mobile.loc[df_mobile['product'] == 'sports_nutrition'])

mobile_accessories_orders = sum(df_mobile.loc[df_mobile['product'] == 'accessories']['target'])
mobile_sneakers_orders = sum(df_mobile.loc[df_mobile['product'] == 'sneakers']['target'])
mobile_clothes_orders = sum(df_mobile.loc[df_mobile['product'] == 'clothes']['target'])
mobile_sn_orders = sum(df_mobile.loc[df_mobile['product'] == 'sports_nutrition']['target'])

mobile_accessories_no_orders = mobile_accessories_total - mobile_accessories_orders
mobile_sneakers_no_orders = mobile_sneakers_total - mobile_sneakers_orders
mobile_clothes_no_orders = mobile_clothes_total - mobile_clothes_orders
mobile_sn_no_orders = mobile_sn_total - mobile_sn_orders

desktop_accessories_total = len(df_desktop.loc[df_desktop['product'] == 'accessories'])
desktop_sneakers_total = len(df_desktop.loc[df_desktop['product'] == 'sneakers'])
desktop_clothes_total = len(df_desktop.loc[df_desktop['product'] == 'clothes'])
desktop_sn_total = len(df_desktop.loc[df_desktop['product'] == 'sports_nutrition'])

desktop_accessories_orders = sum(df_desktop.loc[df_desktop['product'] == 'accessories']['target'])
desktop_sneakers_orders = sum(df_desktop.loc[df_desktop['product'] == 'sneakers']['target'])
desktop_clothes_orders = sum(df_desktop.loc[df_desktop['product'] == 'clothes']['target'])
desktop_sn_orders = sum(df_desktop.loc[df_desktop['product'] == 'sports_nutrition']['target'])

desktop_accessories_no_orders = desktop_accessories_total - desktop_accessories_orders
desktop_sneakers_no_orders = desktop_sneakers_total - desktop_sneakers_orders
desktop_clothes_no_orders = desktop_clothes_total - desktop_clothes_orders
desktop_sn_no_orders = desktop_sn_total - desktop_sn_orders

In [68]:
mobile_accessories_no_orders

34847

In [81]:
#reason this didn't work was that I was comparing across more than 2 categories
contingency_table2 = np.array([
    [mobile_accessories_orders, mobile_accessories_no_orders, mobile_sneakers_orders, mobile_sneakers_no_orders],
    [desktop_accessories_orders, desktop_accessories_no_orders, desktop_sneakers_orders, desktop_sneakers_no_orders]
])

In [78]:
stats.chi2_contingency(contingency_table2)

(577.8538117274697,
 6.371442763301144e-125,
 3,
 array([[  820.58954901, 34754.29400457,  1280.00151974, 36532.11492668],
        [  290.41045099, 12299.70599543,   452.99848026, 12928.88507332]]))

In [76]:
## ian's code

mobile_filter = df['site_version'] == 'mobile'

df3 = df.loc[mobile_filter & time_filter & product_filter].copy()

df3['product'].value_counts()

df_A_m = df3[df3['product'] == 'accessories']
df_B_m = df3[df3['product'] == 'sneakers']

accessories_orders_m = sum(df_A_m['target'])
sneakers_orders_m = sum(df_B_m['target'])

accessories_total_m = len(df_A_m)
sneakers_total_m = len(df_B_m)

accessories_no_order_m = accessories_total_m - accessories_orders_m
sneakers_no_order_m = sneakers_total_m - sneakers_orders_m

accessories_no_order_m + accessories_orders_m

contingency_table_m = np.array([
    [accessories_orders_m, accessories_no_order_m],
    [sneakers_orders_m, sneakers_no_order_m]
])

contingency_table_m

accessory_CR_m, sneaker_CR_m = contingency_table_m[:, 0] / contingency_table_m.sum(axis=1)

print(f'Conversion Rate for accessory banner:\n\t{100*accessory_CR_m:.3f}%')
print(f'Conversion Rate for sneaker banner:\n\t{100*sneaker_CR_m:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(sneaker_CR_m-accessory_CR_m):.3f}%')

Conversion Rate for accessory banner:
	1.734%
Conversion Rate for sneaker banner:
	2.463%

Absolute difference of CR: 0.729%


In [75]:
stats.chi2_contingency(contingency_table_m)

(46.72054613613496,
 8.186616998397816e-12,
 1,
 array([[  748.50638396, 34713.49361604],
        [  800.49361604, 37124.50638396]]))

In [80]:
#garrett's code
df['site_version'].value_counts()

time_filter = df['time'] >= '2019-05-01'

df3 = df.loc[time_filter]

df3.head()

#Null: There is no difference in conversion rate between mobile and desktop versions.

#Alt: There is a difference between mobile and desktop conversion rates.

# Create new dataframes for each site_versions
mobile_filter = df3['site_version'] == 'mobile'
desktop_filter = df3['site_version'] == 'desktop'

# Create new dataframes for each site_versions
df_m = df3.loc[mobile_filter]
df_d = df3.loc[desktop_filter]

# finding total people who bought and didn't buy
# mobile 
mobile_orders = sum(df_m['target'])
mobile_no_orders = len(df_m) - mobile_orders

# desktop
desktop_orders = sum(df_d['target'])
desktop_no_orders = len(df_d) - desktop_orders

print(mobile_orders, mobile_no_orders)
print(desktop_orders, desktop_no_orders)

len(df_m), len(df_d)

# Make a contingency table
contingency_table = np.array([
    [mobile_orders, mobile_no_orders],
    [desktop_orders, desktop_no_orders]    
])
contingency_table

print(stats.chi2_contingency(contingency_table))

# Find the difference in conversion rate
mobile_CR, desktop_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)

print(f'Conversion Rate for mobile banner:\n\t{100*mobile_CR:.3f}%')
print(f'Conversion Rate for desktop banner:\n\t{100*desktop_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(desktop_CR-mobile_CR):.3f}%')

2939 180996
3224 64616
(2063.9893282896182, 0.0, 1, array([[  4502.39859001, 179432.60140999],
       [  1660.60140999,  66179.39859001]]))
Conversion Rate for mobile banner:
	1.598%
Conversion Rate for desktop banner:
	4.752%

Absolute difference of CR: 3.155%


In [84]:
print( contingency_table.sum(axis=1))

[183935  67840]


In [87]:
print(len(df_m), len(df_d))

183935 67840
